In [1]:
import psycopg2
import pandas as pd
from pathlib import Path
import csv


conn = psycopg2.connect(database="f1_halo",
                       host="localhost",
                       user="postgres",
                       password = "postgres",
                       port="5432")

In [2]:
# Store filepaths into variable
driver_csv = Path("data/drivers.csv")
fcyphases_csv = Path("data/fcyphases.csv")
laps_csv = Path("data/laps.csv")
qualifyings_csv = Path("data/qualifyings.csv")
races_csv = Path("data/races.csv")
retirements_csv = Path("data/retirements.csv")
starterfields_csv = Path("data/starterfields.csv")

In [3]:
tables = ["drivers","fcyphases","laps","qualifyings","races","retirements","starterfields"]
for i in tables:
    cursor = conn.cursor()
    query = f'SELECT * from {i};'
    #print(query)
    cursor.execute(query)
    rows = cursor.fetchall()
    df_name = i + "_df"
    globals()[df_name]= pd.DataFrame(rows)
    print(df_name)
    
    
    #csv for headers
    path = f'data/{i}.csv'
    csv_path = Path(path)
    with open(csv_path, 'r', newline='') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the first row
        if len(globals()[df_name].columns) == len(headers):
            globals()[df_name].columns = headers

    #print(df_name)
    print(globals()[df_name].columns)

drivers_df
Index(['id', 'carno', 'initials', 'name'], dtype='object')
fcyphases_df
Index(['id', 'race_id', 'startracetime', 'endracetime', 'startraceprog',
       'endraceprog', 'startlap', 'endlap', 'type'],
      dtype='object')
laps_df
Index(['race_id', 'lapno', 'position', 'driver_id', 'laptime', 'racetime',
       'gap', 'interval', 'compound', 'tireage', 'pitintime',
       'pitstopduration', 'nextcompound', 'startlapprog_vsc', 'endlapprog_vsc',
       'age_vsc', 'startlapprog_sc', 'endlapprog_sc', 'age_sc'],
      dtype='object')
qualifyings_df
Index(['race_id', 'position', 'driver_id', 'q1laptime', 'q2laptime',
       'q3laptime', 'speedtrap'],
      dtype='object')
races_df
Index(['id', 'date', 'season', 'location', 'availablecompounds', 'comment',
       'nolaps', 'nolapsplanned', 'tracklength'],
      dtype='object')
retirements_df
Index(['season', 'driver_id', 'accidents', 'failures'], dtype='object')
starterfields_df
Index(['race_id', 'driver_id', 'team', 'teamcolor', 'eng

In [4]:
fcyphases_df.head()

,id,race_id,startracetime,endracetime,startraceprog,endraceprog,startlap,endlap,type
0,1,1,1069,1604,11.107,14.922,12,15,SC
1,2,3,4157,4899,41.010,45.965,42,46,SC
2,3,6,59,332,0.583,2.897,1,3,SC
3,4,6,2157,2710,25.289,29.916,26,30,SC
4,5,7,38,829,0.372,6.864,1,7,SC


In [5]:
#races_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#races_df.head()

In [6]:
#starterfields_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#starterfields_df.head()

In [7]:
drivers_df.rename(columns = {'id':'driver_id'}, inplace = True)
drivers_df.head()
races_df.rename(columns = {'id':'race_id'}, inplace = True)
races_df.head()

,race_id,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
1,2,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",null,56,56,5543
2,3,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412
3,4,2014-04-20,2014,Shanghai,"A2,A3,I,W",null,54,56,5451
4,5,2014-05-11,2014,Catalunya,"A1,A2,I,W",null,66,66,4655


In [8]:
newdf = pd.merge(drivers_df, starterfields_df)
newdf

,driver_id,carno,initials,name,race_id,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,completedlaps,speedtrap
0,1,44,HAM,Lewis Hamilton,1,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8
1,1,44,HAM,Lewis Hamilton,2,Mercedes,#00D2BE,Mercedes,1,F,1,56,305.9
2,1,44,HAM,Lewis Hamilton,3,Mercedes,#00D2BE,Mercedes,2,F,1,57,313.4
3,1,44,HAM,Lewis Hamilton,4,Mercedes,#00D2BE,Mercedes,1,F,1,54,332.3
4,1,44,HAM,Lewis Hamilton,5,Mercedes,#00D2BE,Mercedes,1,F,1,66,331.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,45,88,KUB,Robert Kubica,117,Williams,#192c4e,Mercedes,20,F,19,50,299.3
2475,45,88,KUB,Robert Kubica,118,Williams,#192c4e,Mercedes,20,F,18,69,352
2476,45,88,KUB,Robert Kubica,119,Williams,#192c4e,Mercedes,19,DNF,19,31,303.7
2477,45,88,KUB,Robert Kubica,120,Williams,#192c4e,Mercedes,19,F,16,70,313.5


In [9]:
newdf=pd.merge(newdf, races_df)

In [10]:
newdf=newdf.set_index(['season','race_id'])
newdf

driver_id  carno initials              name      team  \
season race_id                                                          
2014   1                1     44      HAM    Lewis Hamilton  Mercedes   
       1                2      3      RIC  Daniel Ricciardo   RedBull   
       1                3      6      ROS      Nico Rosberg  Mercedes   
       1                4     20      MAG   Kevin Magnussen   McLaren   
       1                5     14      ALO   Fernando Alonso   Ferrari   
...                   ...    ...      ...               ...       ...   
2019   121             40     16      LEC   Charles Leclerc   Ferrari   
       121             42      4      NOR      Lando Norris   McLaren   
       121             43     23      ALB   Alexander Albon   RedBull   
       121             44     63      RUS    George Russell  Williams   
       121             45     88      KUB     Robert Kubica  Williams   

               teamcolor enginemanufacturer  gridposition status  \
season race_id                                                     
2014   1         #00D2BE           Mercedes             1    DNF   
       1         #1E41FF            Renault             2     DQ   
       1         #00D2BE           Mercedes             3      F   
       1         #FF8700           Mercedes             4      F   
       1         #DC0000            Ferrari             5      F   
...                  ...                ...           ...    ...   
2019   121       #DC0000            Ferrari             3      F   
       121       #FF8700            Renault             6      F   
       121       #1E41FF              Honda             5      F   
       121       #192c4e           Mercedes            18      F   
       121       #192c4e           Mercedes            19      F   

                resultposition  completedlaps speedtrap        date  \
season race_id                                                        
2014   1                    19              2     252.8  2014-03-16   
       1                    22             57     292.7  2014-03-16   
       1                     1             57     299.1  2014-03-16   
       1                     2             57     316.9  2014-03-16   
       1                     4             57     304.5  2014-03-16   
...                        ...            ...       ...         ...   
2019   121                   3             55     313.4  2019-12-01   
       121                   8             54     330.4  2019-12-01   
       121                   6             55     330.7  2019-12-01   
       121                  17             54     329.3  2019-12-01   
       121                  19             53     317.8  2019-12-01   

                 location availablecompounds comment  nolaps  nolapsplanned  \
season race_id                                                                
2014   1        Melbourne          A2,A3,I,W    null      57             58   
       1        Melbourne          A2,A3,I,W    null      57             58   
       1        Melbourne          A2,A3,I,W    null      57             58   
       1        Melbourne          A2,A3,I,W    null      57             58   
       1        Melbourne          A2,A3,I,W    null      57             58   
...                   ...                ...     ...     ...            ...   
2019   121      YasMarina       A4,A6,A7,I,W    null      55             55   
       121      YasMarina       A4,A6,A7,I,W    null      55             55   
       121      YasMarina       A4,A6,A7,I,W    null      55             55   
       121      YasMarina       A4,A6,A7,I,W    null      55             55   
       121      YasMarina       A4,A6,A7,I,W    null      55             55   

                tracklength  
season race_id               
2014   1               5303  
       1               5303  
       1               5303  
       1               5303  
       1               5303  
...                

In [11]:
newdf_refined=newdf[['date','location','driver_id','name','team','enginemanufacturer','resultposition','gridposition','status','completedlaps','comment','nolaps','tracklength']]
newdf_refined

date   location  driver_id              name      team  \
season race_id                                                                 
2014   1        2014-03-16  Melbourne          1    Lewis Hamilton  Mercedes   
       1        2014-03-16  Melbourne          2  Daniel Ricciardo   RedBull   
       1        2014-03-16  Melbourne          3      Nico Rosberg  Mercedes   
       1        2014-03-16  Melbourne          4   Kevin Magnussen   McLaren   
       1        2014-03-16  Melbourne          5   Fernando Alonso   Ferrari   
...                    ...        ...        ...               ...       ...   
2019   121      2019-12-01  YasMarina         40   Charles Leclerc   Ferrari   
       121      2019-12-01  YasMarina         42      Lando Norris   McLaren   
       121      2019-12-01  YasMarina         43   Alexander Albon   RedBull   
       121      2019-12-01  YasMarina         44    George Russell  Williams   
       121      2019-12-01  YasMarina         45     Robert Kubica  Williams   

               enginemanufacturer  resultposition  gridposition status  \
season race_id                                                           
2014   1                 Mercedes              19             1    DNF   
       1                  Renault              22             2     DQ   
       1                 Mercedes               1             3      F   
       1                 Mercedes               2             4      F   
       1                  Ferrari               4             5      F   
...                           ...             ...           ...    ...   
2019   121                Ferrari               3             3      F   
       121                Renault               8             6      F   
       121                  Honda               6             5      F   
       121               Mercedes              17            18      F   
       121               Mercedes              19            19      F   

                completedlaps comment  nolaps  tracklength  
season race_id                                              
2014   1                    2    null      57         5303  
       1                   57    null      57         5303  
       1                   57    null      57         5303  
       1                   57    null      57         5303  
       1                   57    null      57         5303  
...                       ...     ...     ...          ...  
2019   121                 55    null      55         5554  
       121                 54    null      55         5554  
       121                 55    null      55         5554  
       121                 54    null      55         5554  
       121                 53    null      55         5554  

[2479 rows x 13 columns]

In [12]:
meta_df=pd.merge(newdf, retirements_df)
meta_df

,driver_id,carno,initials,name,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,...,date,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength,season,accidents,failures
0,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2014,0,3
1,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2015,0,1
2,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2016,1,1
3,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2017,0,0
4,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,...,2014-03-16,Melbourne,"A2,A3,I,W",null,57,58,5303,2018,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14869,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2015,null,null
14870,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2016,null,null
14871,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2017,null,null
14872,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,...,2019-12-01,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,2018,null,null


In [13]:
meta_df=meta_df.set_index(['season','date'])
meta_df

,,driver_id,carno,initials,name,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,completedlaps,speedtrap,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength,accidents,failures
season,date,,,,,,,,,,,,,,,,,,,,
2014,2014-03-16,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8,Melbourne,"A2,A3,I,W",null,57,58,5303,0,3
2015,2014-03-16,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8,Melbourne,"A2,A3,I,W",null,57,58,5303,0,1
2016,2014-03-16,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8,Melbourne,"A2,A3,I,W",null,57,58,5303,1,1
2017,2014-03-16,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8,Melbourne,"A2,A3,I,W",null,57,58,5303,0,0
2018,2014-03-16,1,44,HAM,Lewis Hamilton,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8,Melbourne,"A2,A3,I,W",null,57,58,5303,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019-12-01,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,53,317.8,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,null,null
2016,2019-12-01,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,53,317.8,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,null,null
2017,2019-12-01,45,88,KUB,Robert Kubica,Williams,#192c4e,Mercedes,19,F,19,53,317.8,YasMarina,"A4,A6,A7,I,W",null,55,55,5554,null,null
